This Jupyter Notebook will guide you on how to make a age detector using OpenCV and Deep Learning

In [1]:
import numpy as np
import cv2
import os


In [2]:
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)",
	"(38-43)", "(48-53)", "(60-100)"]


Load our face-detector which is taken from the opencv repo.

In [3]:
prototxtPath = os.path.join("face_detector/deploy.prototxt")
weightsPath = os.path.join("face_detector/res10_300x300_ssd_iter_140000.caffemodel")
faceNet = cv2.dnn.readNet(prototxtPath, weightsPath)

Load our age-detector model

In [4]:
prototxtPath = os.path.join( "age_detector/age_deploy.prototxt")
weightsPath = os.path.join("age_detector/age_net.caffemodel")
ageNet = cv2.dnn.readNet(prototxtPath, weightsPath)

In [5]:
img = cv2.imread("images/katrina-kaif.jpg")
image = cv2.resize(img, (300,300))
(h, w) = image.shape[:2]
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300),
	(104.0, 177.0, 123.0))


In [6]:
print("[INFO] computing face detections...")
faceNet.setInput(blob)
detections = faceNet.forward()

[INFO] computing face detections...


In [7]:
for i in range(0, detections.shape[2]):
	confidence = detections[0, 0, i, 2]
	if confidence > 0.5:
		box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
		(startX, startY, endX, endY) = box.astype("int")

		face = image[startY:endY, startX:endX]
		faceBlob = cv2.dnn.blobFromImage(face, 1.0, (227, 227),
			(78.4263377603, 87.7689143744, 114.895847746),
			swapRB=False)


		ageNet.setInput(faceBlob)
		preds = ageNet.forward()
		i = preds[0].argmax()
		age = AGE_BUCKETS[i]
		ageConfidence = preds[0][i]

		text = "{}: {:.2f}%".format(age, ageConfidence * 100)
		print("[AGE] {}".format(text))

		y = startY - 10 if startY - 10 > 10 else startY + 10
		cv2.rectangle(image, (startX, startY), (endX, endY),
			(0, 0, 255), 2)
		cv2.putText(image, text, (startX, y),
			cv2.FONT_HERSHEY_SIMPLEX, 0.45, (0, 0, 255), 2)




[AGE] (25-32): 94.46%


In [ ]:
cv2.imshow("Image", image)
cv2.waitKey(0)